In [ ]:
import sys
path = '/Users/xiehao/Desktop/workspace/X2AI/sugar_ai/'
if path not in sys.path:
    sys.path.append(path)

from IPython.display import Markdown
import pandas as pd
from DataBuilder.hisugar.crawler import HigSugarCrawler
from base import DBFile, DBSQL
from AIBots.SentimentalBot.robot import SentimentalBot

# 设置参数
table = "aisugar_hisugar"
start_date = "2023-03-20"
end_date = "2023-04-01"

# # 爬取数据
# HigSugarCrawler(
#     start_date=start_date,
#     end_date=end_date,
#     db=DBSQL()
# ).crawl()

# # 同步数据: 从 SQL数据库 同步到 文件数据库
# df = DBSQL().read_data(table=table, start_date=start_date, end_date=end_date)
# DBFile().save_data(df=df, table=table)

# 同步数据：从BigQuant 同步到 文件数据库
# df = pd.read_parquet("data.parquet", engine='pyarrow')
# DBFile().save_data(df, "aisugar_hisugar")

# AI 研究
bot = SentimentalBot(db=DBFile(), start_date=start_date, end_date=end_date)
reports = bot.analyzing()

In [6]:
user_prompt = ""
report_prompt = """
##  文章标题: {title}
* 发布时间: {publish_report}
* 内容总结: {summary}
* 个人观点: {suggestion}
"""
reports = [i for i in reports if i["summary"] != "" and i["opinion"] != ""]
for i, report in enumerate(reports):
    report_str = report_prompt.format(
        title=report["title"],
        summary=report["summary"],
        suggestion=report["opinion"],
        publish_report=report["date"],
    )
    user_prompt += report_str
    user_prompt += "\n\n"

# AI分析
system_prompt = bot.read_md(bot.filepath_prompt+"researcher.md")
report = bot.api_deepseek(user_prompt, system_prompt)

# 报告保存
bot.save_md(bot.filepath_save, "sentimental", report)